In [65]:
import pandas as pd
import time
from collections import defaultdict, Counter
from itertools import product, chain
import numpy as np

In [30]:
g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
                header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])
g.head()

,cust_id,age_gr,mtype_sec,n_attend
0,11168,50-54,RUNION,1
1,11171,50-54,RUNION,1
2,11179,60-64,FOLK,1
3,11179,60-64,AFL,5
4,11181,55-59,RUNION,24


In [77]:
mtypes_over_years = (pd.read_csv("data/ooo.csv.gz", encoding='latin-1', sep='\t')
                    .applymap(lambda x: x.split(",") if isinstance(x,str) else x)
                    )

In [78]:
mtypes_over_years.head()

,cust_id,00-17,18-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-84
0,11171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN
1,11284,NaN,NaN,NaN,NaN,NaN,[HIPHOP],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11228,NaN,NaN,NaN,NaN,NaN,"[ALTERN, LEAGUE]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[RUNION, LEAGUE]",[LEAGUE],NaN,NaN,NaN,NaN
4,11197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN


Create feature for the shortterm customers. These will be plain binary features showing whother a customer purchesed a particular mtype.

In [101]:
f = defaultdict(lambda: defaultdict(int))  # {cust_id: {"AFL->BALLET": 1, "TENNIS->COMEDY": 1}}

In [102]:
v = mtypes_over_years.T

In [104]:
for c in range(v.shape[1]):
    s = v[c][v[c].notnull()]
    if len(s) == 2:
        f[s[0]] = {"age_gr": s.index[1], "@"+s[1][0]: 1}
    elif len(s) == 3:
        f[s[0]] = {"age_gr": s.index[-1]}
        f[s[0]].update({"@"+"->".join(tp): 1 for tp in product(s[1], s[2])})

KeyboardInterrupt: 

In [29]:
from multiprocessing import Pool

def xf_subdf(d0):
    """
    here d0 is a DATA FRAME of the form
    cust_id
    11196    [30-34, 35-39]
    11391    [60-64, 65-69]
    """

    f = defaultdict(lambda: defaultdict(int))
    
    for r in d0.itertuples():
        # make "transition" feature
        f[r.cust_id] = {"->".join(tp): 1 for tp in product(*[g_purch_byage[(r.cust_id, ag)] 
                                                        for ag in d0[r.cust_id]])}
        # add age group 
        f[r.cust_id].update({"age_gr": g_age_gr_each_customer[r.cust_id][-1]})
    
    # create a data frame from dict and return it
    return pd.DataFrame.from_dict(f, orient='index').fillna(0)

t0 = time.time()

def xf_df(d1):
    
    df_split = [d1.iloc[:3000], d1.iloc[3001:6000]]
    
    pool = Pool(2)
    
    df = pd.concat(pool.map(xf_subdf, df_split))
    
    pool.close()
    pool.join()
    
    return df

f1 = xf_df(g_age_gr_each_customer_longterm.reset_index())

print("elapsed time: {:.0f} m {:.0f} s".format(*divmod(time.time()-t0, 60)))

KeyError: 138115

Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(d1.drop("age_gr", axis=1), d1["age_gr"], 
                                                    test_size=0.3, random_state=94, stratify=d1["age_gr"])

In [ ]:
print("created training data - features {} and target {}".format(X_train.shape, y_train.shape))

In [ ]:
gnb = BernoulliNB()
gnb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

print("accuracy: {:.3f}".format(accuracy_score(y_test, svc.predict(X_test))))
scores = cross_val_score(svc, X_train, y_train, cv=5)


#print("accuracy: {:.3f}".format(accuracy_score(y_test, gnb.predict(X_test))))
# scores = cross_val_score(gnb, X_train, y_train, cv=5)
print("cross-validation scores: {}, {}, {}, {}, {}".format(*scores))